# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [15]:
# Save the Folium map to HTML
site_map.save("site_map.html")

# Display it in the notebook
import IPython
IPython.display.IFrame("site_map.html", width=800, height=500)
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [24]:
import pandas as pd
import folium
from folium.features import DivIcon
import io
from js import fetch  # for fetching CSV in browser-based notebooks

# Fetch the CSV dataset
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Get one row per launch site
launch_sites_df = spacex_df.groupby('Launch Site', as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# NASA Johnson Space Center coordinates for centering the map
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch site to add Circle + Marker
for idx, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Circle for launch site
    circle = folium.Circle(
        location=coord,
        radius=1000,
        color='#000000',
        fill=True,
        fill_opacity=0.5
    ).add_child(folium.Popup(launch_site_name))
    
    # Marker with label
    marker = folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{launch_site_name}</b></div>',
        )
    )
    
    # Add circle and marker to map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Render the map inline (works in any environment)
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   All launch sites are near the Equator (28–34° N) for efficiency.
*   Are all launch sites in very close proximity to the coast?
*   All launch sites are very close to the ocean for safety reasons.

Also please try to explain your findings.
Launching closer to the Equator provides a rotational speed boost from Earth’s rotation, making it more fuel-efficient for rockets.
Launching over water is safer:
If a rocket fails, debris falls into the ocean instead of populated areas.
Coastal locations also allow easier launch azimuths for different orbits.


In [ ]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [25]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [26]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [27]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [30]:
import pandas as pd
import folium
from folium.features import DivIcon
from folium.plugins import MarkerCluster
import io
from js import fetch  # for fetching CSV in browser-based notebooks

# -------------------------------
# 1. Load the dataset
# -------------------------------
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# -------------------------------
# 2. Initialize the map
# -------------------------------
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# -------------------------------
# 3. Add launch sites as circles + labels (Task 1)
# -------------------------------
launch_sites_df = spacex_df.groupby('Launch Site', as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

for idx, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    # Circle
    circle = folium.Circle(
        location=coord,
        radius=1000,
        color='#000000',
        fill=True,
        fill_opacity=0.5
    ).add_child(folium.Popup(site_name))
    
    # Label marker
    marker = folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site_name}</b></div>'
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

# -------------------------------
# 4. Task 2: Add success/failure markers
# -------------------------------
# Create a new column for marker color based on success/failure
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Create MarkerCluster
marker_cluster = MarkerCluster()

# Add markers to the cluster
for index, record in spacex_df.iterrows():
    coord = [record['Lat'], record['Long']]
    popup_text = f"Launch Site: {record['Launch Site']}<br>Success: {record['class']}"
    
    marker = folium.Marker(
        location=coord,
        popup=popup_text,
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket', prefix='fa')
    )
    
    marker_cluster.add_child(marker)

# Add cluster to the map
site_map.add_child(marker_cluster)

# -------------------------------
# 5. Display the map inline
# -------------------------------
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [33]:
import pandas as pd
import folium
from folium.features import DivIcon
from folium.plugins import MarkerCluster, MousePosition
import io
from js import fetch  # for browser-based notebooks
import math

# -------------------------------
# Haversine distance function
# -------------------------------
def haversine(coord1, coord2):
    # Coordinates in decimal degrees (lat, lon)
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    c = 2*math.asin(math.sqrt(a))
    
    # Radius of Earth in km
    r = 6371
    return c * r

# -------------------------------
# 1. Load dataset
# -------------------------------
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Keep relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# -------------------------------
# 2. Initialize map
# -------------------------------
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# -------------------------------
# 3. Add launch sites (Task 1)
# -------------------------------
launch_sites_df = spacex_df.groupby('Launch Site', as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

for idx, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    circle = folium.Circle(
        location=coord,
        radius=1000,
        color='#000000',
        fill=True,
        fill_opacity=0.5
    ).add_child(folium.Popup(site_name))
    
    marker = folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site_name}</b></div>'
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

# -------------------------------
# 4. Add success/failure markers (Task 2)
# -------------------------------
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
marker_cluster = MarkerCluster()

for index, record in spacex_df.iterrows():
    coord = [record['Lat'], record['Long']]
    popup_text = f"Launch Site: {record['Launch Site']}<br>Success: {record['class']}"
    
    marker = folium.Marker(
        location=coord,
        popup=popup_text,
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket', prefix='fa')
    )
    
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)

# -------------------------------
# 5. Add MousePosition plugin (Task 3)
# -------------------------------
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter
)

site_map.add_child(mouse_position)

# -------------------------------
# 6. Example distance calculations using Haversine
# -------------------------------
points_of_interest = {
    "Railway": (28.590, -80.600),
    "Lake": (28.565, -80.590)
}

distance_results = []
for idx, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    site_coord = (row['Lat'], row['Long'])
    for poi_name, poi_coord in points_of_interest.items():
        distance_km = haversine(site_coord, poi_coord)
        distance_results.append({
            "Launch Site": site_name,
            "POI": poi_name,
            "Distance (km)": round(distance_km, 2)
        })

distance_df = pd.DataFrame(distance_results)
print(distance_df)

# -------------------------------
# 7. Display the map inline
# -------------------------------
site_map

    Launch Site      POI  Distance (km)
0   CCAFS LC-40  Railway           3.79
1   CCAFS LC-40     Lake           1.27
2  CCAFS SLC-40  Railway           3.74
3  CCAFS SLC-40     Lake           1.30
4    KSC LC-39A  Railway           4.94
5    KSC LC-39A     Lake           5.63
6   VAFB SLC-4E  Railway        3822.69
7   VAFB SLC-4E     Lake        3824.58


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [34]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [35]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [39]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [40]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [41]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [42]:
from math import sin, cos, sqrt, atan2, radians
import folium
from folium.features import DivIcon

# -------------------------------
# Distance function
# -------------------------------
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of earth in km
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# -------------------------------
# Example launch site
# -------------------------------
launch_site_coord = [28.563, -80.576]  # CCAFS SLC-40

# -------------------------------
# Example points of interest (POI)
# -------------------------------
points_of_interest = {
    "Coastline": [28.56367, -80.57163],
    "Railway": [28.590, -80.600],
    "Highway": [28.565, -80.585]
}

# -------------------------------
# Initialize map
# -------------------------------
site_map = folium.Map(location=launch_site_coord, zoom_start=12)

# Launch site marker
folium.Marker(
    location=launch_site_coord,
    popup="Launch Site",
    icon=folium.Icon(color='green', icon='rocket', prefix='fa')
).add_to(site_map)

# -------------------------------
# Add markers and lines for POIs
# -------------------------------
for name, coord in points_of_interest.items():
    # Calculate distance
    distance_km = calculate_distance(launch_site_coord[0], launch_site_coord[1], coord[0], coord[1])
    
    # Create marker with distance label
    folium.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{name}: {distance_km:.2f} KM</b></div>'
        )
    ).add_to(site_map)
    
    # Draw line between launch site and POI
    folium.PolyLine(
        locations=[launch_site_coord, coord],
        color='blue',
        weight=2,
        dash_array='5'
    ).add_to(site_map)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Yes it is near enough for transport/logistics but safely distant
*   Are launch sites in close proximity to highways?
*   Close for access, but not directly on launch site.
*   Are launch sites in close proximity to coastline?
*   Very close (few km) for safety and launch trajectory over water.
*   Do launch sites keep certain distance away from cities?
*   Kept at safe distance to minimize risk to population

Also please try to explain your findings.
Launch sites are strategically positioned considering safety, logistics, and trajectory requirements.

Close to coastlines for over-water launches.

Sufficient distance from cities and sensitive areas.

Accessibility is ensured through nearby highways and railways without compromising safety.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.
!pip install dash dash-bootstrap-components
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("SpaceX Launch Records Dashboard"),
            dcc.Dropdown(
                id='site-dropdown',
                options=[{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
                value='All Sites',
                clearable=False
            )
        ])
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='success-pie-chart')),
        dbc.Col(dcc.Graph(id='payload-scatter-chart'))
    ])
])

# Callbacks to update charts will go here

if __name__ == '__main__':
    app.run_server(debug=True)


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
